# 7. Data Wrangling: Clean, Transform, Merge, Reshape
o 데이터 분석의 절차
 - 1) 문제의 정의 
 - 2) 데이터 준비: Extract(Data Gathering) - Transform (Cleaning, 변형, 정규화) - Loading 
 - 3) 데이터 탐구: 통계적 탐구 
 - 4) 예측 모델링: 모델생성, 검증,
 - 5) 결과 시각화: 시각화 및 해석
 - 6) 솔루션 보고 및 배포
 
o 데이터 분석 프로세스
 - 데이터 준비에 가장 많은 자원(시간, 비용, 인력)이 소모됨.
 
o pandas에서는 다양한 ETL 기능을 제공함.
 

In [19]:
from __future__ import division  # python 2에서 3에서 사용될 / 연산자의 사용법을 미리 사용 
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas
import pandas as pd
np.set_printoptions(precision=4, threshold=500)
pd.options.display.max_rows = 100

In [20]:
%matplotlib inline

## 7.1 Combining and merging data sets
o pandas 객체에 저장된 데이터는 여러 내장 함수를 이용해 합치기 가능
 - pandas.merge: 하나 이상의 키를 기준으로 DataFrame의 로우를 합친다. SQL이나 다른 관계형 데이터 베이스의 join 연산과 유사
   (DataFrame.join 과 비교 예정) 
 - pandas.concat은 하나의 축을 따라 객체를 이어붙임
 - combine_first 인스턴스 메서드는 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채울 수 있도록 한다.

### 7.1.1 Database-style DataFrame merges
o merge나 join 연산은 관계형 데이터베이스의 핵심적인 연산으로, 키를 하나 이상 사용해서 데이터 집합의 로우를 합침
 - merge key를 지정하지 않은 경우, 동일 컬럼명을 조인 키로 활용함. 

In [21]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df2 = DataFrame({'key': ['a', 'b', 'd'],
                 'data2': range(3)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [22]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [23]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [24]:
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [25]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df4 = DataFrame({'rkey': ['a', 'b', 'd'],
                 'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [26]:
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [27]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                 'data1': range(6)})
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                 'data2': range(5)})

In [28]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [29]:
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [30]:
pd.merge(df1, df2, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [31]:
pd.merge(df1, df2, how='inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [32]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [33]:
pd.merge(left,right)

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,bar,one,3,6


In [34]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [35]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


#### o merge 함수 인자 목록
 - left	머지하려는 DataFrame 중 왼쪽에 위치한 DataFrame
 - right	머지하려는 DataFrame 중 오른쪽에 위치한 DataFrame
 - how	조인방법. 'inner', 'outer', 'left', 'right'. 기본값은 inner
 - on	조인하려는 로우 이름. 반드시 두 DataFrame 객체 모두에 있는 이름이어야 한다. 만약 명시되지 않고 다른 조인 키도 주어지지 않으면 left와 right에서 공통되는 칼럼을 조인 키로 사용한다.
 - left_on	조인 키로 사용할 left DataFrame의 칼럼
 - right_on	조인 키로 사용할 right DataFrame의 칼럼
 - left_index	조인 키로 사용할 left DataFrame의 색인 로우(다중 색인일 경우의 키)
 - right_index	조인 키로 사용할 right DataFrame의 색인 로우(다중 색인일 경우의 키)
 - sort	조인 키에 따라 병합된 데이터를 사전 순으로 정렬. 기본값은 True. 대용량 데이터의 경우 False라면 성능상의 이득을 얻을 수도 있다.
 - suffixes	칼럼 이름이 겹칠 경우 각 칼럼 이름 뒤에 붙일 문자열의 튜플. 기본값은 ('_x', '_y'). 만약 'data'라는 칼럼 이름이 양쪽 DataFrame에 같이 존재하면 결과에서는 'data_x', 'data_y'로 나타난다.
 - copy	False일 경우 예외적인 경우에 결과로 데이터가 복사되지 않도록 한다. 기본값은 항상 복사가 이루어진다.

### 7.1.2 Merging on index

o 컬럼대신 index를 조인기준으로 활용하여 Merge 가능


In [36]:
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                  'value': range(6)})
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [37]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [38]:
right1

,group_val
a,3.5
b,7.0


In [39]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [40]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [41]:
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                   'key2': [2000, 2001, 2002, 2001, 2002],
                   'data': np.arange(5.)})
righth = DataFrame(np.arange(12).reshape((6, 2)),
                   index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                          [2001, 2000, 2000, 2000, 2001, 2002]],
                   columns=['event1', 'event2'])
lefth

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


In [42]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [43]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [44]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


In [45]:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.],[7.,8.]], index=['a', 'c', 'e','d'],
                 columns=['Ohio', 'Nevada'])
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                   index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])

In [46]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0
d,7.0,8.0


In [47]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [48]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,7.0,8.0,11.0,12.0
e,5.0,6.0,13.0,14.0


#### o DataFrame.join 
 - join 연산은 기본적으로 index를 기준으로 합병한다.
 - 기본적으로 호출하는 DataFrame을 outer join ( = Left Outer JOIN) 한다. 
 - 유사한(동일한) 색인구조를 갖을 경우 편리하게 사용 가능

In [49]:
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0
d,7.0,8.0,11.0,12.0


In [50]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,7.0,8.0,11.0,12.0
e,5.0,6.0,13.0,14.0


In [51]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [52]:
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                    index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [53]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0
d,7.0,8.0


In [54]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [69]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
d,7.0,8.0,11.0,12.0,NaN,NaN


##### o outer : 3개 dataframe을 모두 보존  

In [55]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,7.0,8.0,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


### 7.1.3 Concatenating along an axis
#### o 또 다른 합병의 종류: concatenating(이어붙이기), binding(연결), stacking(적층)
#### o concatenating
 - NumPy의 concatenate()

In [56]:
arr = np.arange(12).reshape((3, 4))

In [57]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [58]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [92]:
s1 = Series([0, 1], index=['a', 'b'])
# s1 = Series([0, 1], index=['a', 'c'])
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = Series([5, 6], index=['f', 'g'])

#### o Pandas.concat(Series, ...)
 - 디폴트 axis=0 을 기준으로 연결하여 붙임.(join 하지 않음. 단순 연결) 

In [76]:
pd.concat([s1, s2, s3])

a    0
c    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [69]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [70]:
s4 = pd.concat([s1 * 5, s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [71]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


#### o pd.concat( )의 옵션
 - 참고) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html
  - return: 
  - When concatenating all Series along the index (axis=0), a Series is returned. 
  - When objs contains at least one DataFrame, a DataFrame is returned. 
  - When concatenating along the columns (axis=1), a DataFrame is returned.

In [81]:
s1

a    0
c    1
dtype: int64

In [82]:
s4

a    0
b    5
f    5
g    6
dtype: int64

In [86]:
pd.concat([s1, s4], axis=1)  # axis=1 일 경우에만,join 옵션(디폴트=outer)이 의미 있음. 
# pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0.0,0.0
b,NaN,5.0
c,1.0,NaN
f,NaN,5.0
g,NaN,6.0


- join_axes 조인할 행의 축의 값을  지정

In [87]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,1.0,NaN
b,NaN,5.0
e,NaN,NaN


- keys: Series를 이어붙일 경우, 다단계 색인을 통하여 소속 Series를 표기

In [88]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])

In [89]:
result

one    a    0
       c    1
two    a    0
       c    1
three  f    5
       g    6
dtype: int64

In [90]:
# Much more on the unstack function later
result.unstack()

,a,c,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


- keys : axis=1 이면, 
 - Series를 concat 한 결과는 DataFrame이 되고, 
 - keys 옵션은 결과 DataFrame의 컬럼이 된다.

In [96]:
# pd.concat([s1, s2, s3])  # s1,s2,s3 확인 
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


#### o pd.concat(DataFrame+) 
 - Series와 동일하게 적용
 - keys: 기존 df에 컬럼명이 있으므로, 다단계 컬럼으로... 

In [100]:
df1 = DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                columns=['one', 'two'])
df2 = DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                columns=['three', 'four'])
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])
# pd.concat([df1, df2], axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [102]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [103]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [104]:
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [105]:
df1

,a,b,c,d
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221


In [106]:
df2

,b,d,a
0,0.274992,0.228913,1.352917
1,0.886429,-2.001637,-0.371843


- ignore_index = TRUE : index가 불필요한 경우
  - 참고) 디폴트 값: axix=0 

In [109]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,1.352917,0.274992,NaN,0.228913
4,-0.371843,0.886429,NaN,-2.001637


### 7.4.1 겹치는 데이터 합치기

In [110]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b = Series(np.arange(len(a), dtype=np.float64),
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan

In [111]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [112]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

#### o np.where(condition[, x, y] ) 
 - 참고: https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.where.html
 - condition, x, y 는 array-like 객체
 - condition이 TRUE이면 x, 아니면 y 선택
 - return: ndarray or tuple of ndarrays

In [113]:
np.where(pd.isnull(a), b, a) # a가 null 인 곳만, b가 리턴됨 

array([0. , 2.5, 2. , 3.5, 4.5, nan])

#### Series.combine_first( ) 
 - 참고) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.combine_first.html
 - 호출 Series의 값을 우선하고 NULL인 경우만, 파라미터 Series의 값으로 대체
 - 색인값으로 정렬 

In [130]:
b[:-2]

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64

In [131]:
a[2:]

d    3
c    4
dtype: int64

In [125]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [126]:
df1 = DataFrame({'a': [1., np.nan, 5., np.nan],
                 'b': [np.nan, 2., np.nan, 6.],
                 'c': range(2, 18, 4)})
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.],
                 'b': [np.nan, 3., 4., 6., 8.]})
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 7.2 재형성과 피봇(Reshaping and pivoting)

### 7.2.1 Reshaping with hierarchical indexing
#### o stack(level=-1, dropna=True) 
 - 데이터 컬럼을 가장 안쪽의 색인으로 피봇 또는 회전시킨다. 
 - dropna=TRUE 이므로, 누락된 값은 자동으로 사라짐.(참고: 아래)
 - 참고) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.stack.html

#### o unstack(level=-1, fill_value=None): 
 - 로우를 컬럼의 가장안쪽 레벨로 피봇 또는 회전시킨다. 
 -  fill_value=None 이므로, 불일치하는 피봇 색인들은 Nan으로 채워짐.(참고: 아래)
 - 참고) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html


In [136]:
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [137]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [138]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [139]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [140]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [144]:
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two']) # keys가 원래의 Series을 알 수 있도록 색인레벨 부여
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

- unstack()의 경우, povot 레벨의 색인이 일치하지 않으면, Nan값 발생

In [149]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [150]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [151]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

- unstack() : 이동 목표위치는 가장 안쪽 컬럼레벨이다.

In [154]:
df = DataFrame({'left': result, 'right': result + 5},
               columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [155]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [ ]:
df.unstack('state').stack('side')

### 7.2.2 피봇팅으로 데이터 나열방식 바꾸기 (Pivoting "long" to "wide" format)

In [158]:
data = pd.read_csv('ch07/macrodata.csv')
data

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
5,1960.0,2.0,2834.390,1792.9,298.152,460.400,1966.1,29.550,140.2,2.68,5.2,180.671,0.14,2.55
6,1960.0,3.0,2839.022,1785.8,296.375,474.676,1967.8,29.750,140.9,2.36,5.6,181.528,2.70,-0.34
7,1960.0,4.0,2802.616,1788.2,259.764,476.434,1966.6,29.840,141.1,2.29,6.3,182.287,1.21,1.08
8,1961.0,1.0,2819.264,1787.7,266.405,475.854,1984.5,29.810,142.1,2.37,6.8,182.992,-0.40,2.77
9,1961.0,2.0,2872.005,1814.3,286.246,480.328,2014.4,29.920,142.9,2.29,7.0,183.691,1.47,0.81


In [161]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

#### DataFrame.to_records(index=True, convert_datetime64=True)
 - Convert DataFrame to record array. 
 - 참고) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_records.html

#### PeriodIndex.to_timestamp(freq=None, how='start')
 - Cast to DatetimeIndex
 - 참고) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.PeriodIndex.to_timestamp.html
   - freq : string or DateOffset, default ‘D’ for week or longer, ‘S’
otherwise. 
   - how : {‘s’, ‘e’, ‘start’, ‘end’}

In [178]:
data = DataFrame(data.to_records(),
                 columns=pd.Index(['realgdp', 'infl', 'unemp'], name='item'),
                 index=periods.to_timestamp('D', 'end'))
data[:10]

item,realgdp,infl,unemp
date,,,
1959-03-31,2710.349,0.00,5.8
1959-06-30,2778.801,2.34,5.1
1959-09-30,2775.488,2.74,5.3
1959-12-31,2785.204,0.27,5.6
1960-03-31,2847.699,2.31,5.2
1960-06-30,2834.390,0.14,5.2
1960-09-30,2839.022,2.70,5.6
1960-12-31,2802.616,1.21,6.3
1961-03-31,2819.264,-0.40,6.8


In [177]:
# ldata = data.stack().reset_index()#.rename(columns={0: 'value'})
# ldata
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata[:10]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


#### DataFrame.pivot(index=None, columns=None, values=None)
 - dataframe을 지정된 각 값들을 index, column, values로 사용하도록 피봇팅한 결과 산출 
 - 참고) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html

In [179]:
wdata = ldata.pivot('date', 'item', 'value')
wdata[:10]

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2
1960-06-30,0.14,2834.390,5.2
1960-09-30,2.70,2839.022,5.6
1960-12-31,1.21,2802.616,6.3
1961-03-31,-0.40,2819.264,6.8


In [171]:
ldata[:10]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


In [172]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


In [173]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,1.669025
1,1959-03-31,infl,0.000,-0.438570
2,1959-03-31,unemp,5.800,-0.539741
3,1959-06-30,realgdp,2778.801,0.476985
4,1959-06-30,infl,2.340,3.248944
5,1959-06-30,unemp,5.100,-1.021228
6,1959-09-30,realgdp,2775.488,-0.577087
7,1959-09-30,infl,2.740,0.124121
8,1959-09-30,unemp,5.300,0.302614
9,1959-12-31,realgdp,2785.204,0.523772


- values=None이면, 나머지 모든 컬럼이 "values="의 값으로 지정됨. 

In [180]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                
item        infl   realgdp unemp
date                            
1959-03-31  0.00  2710.349   5.8
1959-06-30  2.34  2778.801   5.1
1959-09-30  2.74  2775.488   5.3
1959-12-31  0.27  2785.204   5.6
1960-03-31  2.31  2847.699   5.2

In [181]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


In [184]:
#unstacked = ldata.set_index(['date', 'item'])#.unstack('item')
#unstacked[:7]
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                
item        infl   realgdp unemp
date                            
1959-03-31  0.00  2710.349   5.8
1959-06-30  2.34  2778.801   5.1
1959-09-30  2.74  2775.488   5.3
1959-12-31  0.27  2785.204   5.6
1960-03-31  2.31  2847.699   5.2
1960-06-30  0.14  2834.390   5.2
1960-09-30  2.70  2839.022   5.6

## 7.3 데이터 변형(Data transformation)

### 7.3.1 중복제거(Removing duplicates)

In [199]:
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


#### dataframe.duplicated(subset=None, keep='first')
 - Return boolean Series denoting duplicate rows, optionally only considering certain columns
 - keep 값(first, last, False) 에 따라서, False의 위치를 결정 
   - 참고: keep이므로 duplicate하지 않은 것(keep할 것)으로 판정(false)할 것을 지정, 
   - 따라서, False는 keep할 것이 없으므로, 중볻된 모든 항목이 TRUE
   

In [187]:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [188]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [192]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,4,5
6,two,4,6


In [193]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3


In [200]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2
1,one,1
2,one,2
4,two,3
6,two,4


### 7.3.2 함수나 매핑으로 데이터 변형하기(Transforming data using a function or mapping)
- 필요성: 값에 따라 데이터의 모양 변경 필요

In [201]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
                           'corned beef', 'Bacon', 'pastrami', 'honey ham',
                           'nova lox'],
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [202]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

#### o Series.map(arg, na_action=None)
 - Map values of Series using input correspondence (which can be a dict, Series, or function)
  - dict: key -> value, 
  - Series: index -> value
  - Function: input -> funtion return 

In [205]:
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [206]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 7.3.3 값 치환하기(Replacing values)
#### o Series.replace(to_replace=None, value=None, inplace=False, limit=None, regex=False, method='pad', axis=None)
 - https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.replace.html
 - to_replace : str, regex, list, dict, Series, numeric, or None
 - value : scalar, dict, list, str, regex, default None 

In [209]:
data = Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [210]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [211]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [212]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [213]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 7.3.4 DataFrame 축 색인 이름변경(Renaming axis indexes)
##### o Index.map(mapper)
 - Series와 유사하게, Index에 map 함수 보유 
 - Apply mapper function to an index
   - mapper : callable

In [214]:
data = DataFrame(np.arange(12).reshape((3, 4)),
                 index=['Ohio', 'Colorado', 'New York'],
                 columns=['one', 'two', 'three', 'four'])

In [215]:
data.index.map(str.upper)

Index(['OHIO', 'COLORADO', 'NEW YORK'], dtype='object')

In [217]:
data.index = data.index.map(str.upper)
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


#### o DataFrame.rename(mapper=None, index=None, columns=None, axis=None, copy=True, inplace=False, level=None)
 - https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html
 - Alter axes labels.
 - mapper, index, columns : dict-like or function, optional
  - Function / dict values must be unique (1-to-1). 
  - Labels not contained in a dict / Series will be left as-is.
  - Extra labels listed don’t throw an error.
 - copy : boolean, default True  ==> rename copies and return new DataFrame

In [219]:
# (mapper, index, columns)값이 함수인경우 
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [220]:
# (mapper, index, columns)값이 사전인경우 
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [222]:
# Always returns a reference to a DataFrame
_ = data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data  # Look at inplcace=TRUE

,one,two,three,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


### 7.3.5 개별화와 양자화(Discretization and binning)
- 필요성: 데이터를 분할하거나 그룹화에 대한 필요는 상존

#### o class pandas.Categorical(values, categories=None, ordered=None, dtype=None, fastpath=False)
- https://pandas.pydata.org/pandas-docs/stable/categorical.html
- Represents a categorical variable in classic R / S-plus fashion
- Categoricals can only take on only a limited, and usually fixed, number of possible values (categories)
- All values of the Categorical are either in categories or np.nan.
- Order is defined by the order of the categories, not lexical order of the values
- Parameters:
  - values : list-like
    - The values of the categorical. If categories are given, values not in categories will be replaced with NaN.
  - categories : Index-like (unique), optional
    - The unique categories for this categorical. If not given, the categories are assumed to be the unique values of values.
  - ordered : boolean, (default False)
- 결론적으로, 
  - 1) categories가 주어지면, valeuse는 분류될 값이고, categories는 분류기준을 제공
  - 2) categories가 안 주어지면, valeuse는 분류될 값이자, categories 자체가 됨. 
- Categorical.codes
  - Level codes are an array if integer which are the positions of the real values in the categories array.  - 

In [227]:
# Example 1 of Categorical
pd.Categorical([1, 2, 3, 1, 2, 3])

[1, 2, 3, 1, 2, 3]
Categories (3, int64): [1, 2, 3]

In [228]:
# Example 2 of Categorical
pd.Categorical(['a', 'b', 'c', 'a', 'b', 'c'])

[a, b, c, a, b, c]
Categories (3, object): [a, b, c]

In [230]:
c = pd.Categorical(['a','b','c','a','b','c'], ordered=True, categories=['c', 'b', 'a'])
c

[a, b, c, a, b, c]
Categories (3, object): [c < b < a]

#### o pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False)
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.cut.html
- Return indices of half-open bins to which each value of x belongs.
 - out: Categorical or Series or array of integers
 - bins:ndarray of floats only if retbins is True.
- Parameters
 - x : array-like
   - Input array to be binned. It has to be 1-dimensional.
 - bins : int, sequence of scalars, or IntervalIndex
   - If bins is an int, it defines the number of equal-width bins in the range of x. 
     - However, in this case, the range of x is extended by .1% on each side to include the min or max values of x
   - If bins is a sequence it defines the bin edges allowing for non-uniform bin width
 - right : bool, optional
   - Indicates whether the bins include the rightmost edge or not.
 - labels : array or boolean, default None
   - Used as labels for the resulting bins. Must be of the same length as the resulting bins. 
   - If False, return only integer indicators of the bins.

In [233]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [234]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [235]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [238]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [241]:
# right : 경계포함(대괄호)의 위치를 결정
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [242]:
# lables 
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [261]:
data = np.random.rand(20)
data

array([0.7916, 0.8906, 0.4121, 0.8414, 0.8749, 0.4262, 0.4529, 0.1959,
       0.3079, 0.519 , 0.9444, 0.8903, 0.6352, 0.4665, 0.9858, 0.3782,
       0.8924, 0.4226, 0.7334, 0.8791])

In [262]:
cut = pd.cut(data, 4)
cut

[(0.788, 0.986], (0.788, 0.986], (0.393, 0.591], (0.788, 0.986], (0.788, 0.986], ..., (0.195, 0.393], (0.788, 0.986], (0.393, 0.591], (0.591, 0.788], (0.788, 0.986]]
Length: 20
Categories (4, interval[float64]): [(0.195, 0.393] < (0.393, 0.591] < (0.591, 0.788] < (0.788, 0.986]]

In [263]:
cut.value_counts()

(0.195, 0.393]    3
(0.393, 0.591]    6
(0.591, 0.788]    2
(0.788, 0.986]    9
dtype: int64

In [259]:
mydata=np.arange(10)
mydata

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [260]:
# bins 가 integer일 경우, 1% of range가 min, max에서 확대될 수 있다. see min boundary
mycut = pd.cut(mydata, 4)
mycut

[(-0.009, 2.25], (-0.009, 2.25], (-0.009, 2.25], (2.25, 4.5], (2.25, 4.5], (4.5, 6.75], (4.5, 6.75], (6.75, 9.0], (6.75, 9.0], (6.75, 9.0]]
Categories (4, interval[float64]): [(-0.009, 2.25] < (2.25, 4.5] < (4.5, 6.75] < (6.75, 9.0]]

#### o pandas.qcut(x, q, labels=None, retbins=False, precision=3, duplicates='raise')
- Quantile-based discretization function. 
- Discretize variable into equal-sized buckets based on rank or based on sample quantiles. 
  - quantile: 변위치, 예) 4분위, 10%분위 등
- For example 1000 values for 10 quantiles would produce a Categorical object indicating quantile membership for each data point.
- Parameters:
  - q : integer or array of quantiles, Number of quantiles. Alternately array of quantiles, e.g. [0, .25, .5, .75, 1.]
- cut() vs. qcut()
 - cut(): 구간길기가(bin의 길이) 값을 기준으로 동일
 - qcut(): 변위치를 사용하기 때문에, value_counts()가 동일하도록 구간길기를 지정 (각 bin에 소속되는 갯수가 동일)

In [264]:
data = np.random.randn(1000) # Normally distributed
cats = pd.qcut(data, 4) # Cut into quartiles
cats

[(-0.01, 0.715], (0.715, 3.367], (-0.01, 0.715], (-0.01, 0.715], (-0.01, 0.715], ..., (-0.682, -0.01], (-0.682, -0.01], (-0.682, -0.01], (-0.682, -0.01], (-0.682, -0.01]]
Length: 1000
Categories (4, interval[float64]): [(-3.55, -0.682] < (-0.682, -0.01] < (-0.01, 0.715] < (0.715, 3.367]]

In [265]:
pd.value_counts(cats)

(0.715, 3.367]     250
(-0.01, 0.715]     250
(-0.682, -0.01]    250
(-3.55, -0.682]    250
dtype: int64

In [267]:
# 변위치를 직접 지정하는 경우 
cats=pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
cats

[(-0.01, 1.326], (-0.01, 1.326], (-0.01, 1.326], (-0.01, 1.326], (-0.01, 1.326], ..., (-1.242, -0.01], (-1.242, -0.01], (-1.242, -0.01], (-1.242, -0.01], (-1.242, -0.01]]
Length: 1000
Categories (4, interval[float64]): [(-3.55, -1.242] < (-1.242, -0.01] < (-0.01, 1.326] < (1.326, 3.367]]

In [270]:
pd.value_counts(cats)

(-0.01, 1.326]     400
(-1.242, -0.01]    400
(1.326, 3.367]     100
(-3.55, -1.242]    100
dtype: int64

### 7.3.6 특이값 찾아내고 제외하기 (Detecting and filtering outliers)

In [271]:
np.random.seed(12345)
data = DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [274]:
col = data[3]

In [275]:
col[np.abs(col) > 3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [276]:
# 3이 넘어가는 값이 1개 이상 존재하는 행을 선택 
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [278]:
data[np.abs(data) > 3] = np.sign(data) * 3 # numpy.sign()  사인함수 값
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


### 7.3.7 Permutation and random sampling
#### o numpy.random.permutation(x)
- Randomly permute a sequence, or return a permuted range.
- If x is a multi-dimensional array, it is only shuffled along its first index.
- parameters
 - x : int or array_like
   - If x is an integer, randomly permute np.arange(x).
   - f x is an array, make a copy and shuffle the elements randomly.
- returns 
 - out : ndarray 
   - Permuted sequence or array range.

In [279]:
df = DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([1, 0, 2, 3, 4])

In [280]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


#### o DataFrame.take(indices, axis=0, convert=None, is_copy=True, **kwargs)
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.take.html
- Return the elements in the given positional indices along an axis
- Parameters:
 - indices : array-like
 - axis : int, default 0

In [282]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


- 일부만 선택하고자 하는 경우

In [284]:
df.take(np.random.permutation(len(df))[:3])

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
0,0,1,2,3


- 주어진 값으로만 치환을 통해 샘플을 생성하는 방법
 - 예) 주어진 bag의 값을, 10개의 샘플을 생성 

In [285]:
bag = np.array([5, 7, -1, 6, 4])
sampler = np.random.randint(0, len(bag), size=10)

In [286]:
sampler

array([2, 2, 0, 3, 0, 4, 1, 1, 2, 3])

In [288]:
# numpy.take(a, indices, axis=None, out=None, mode='raise')  - df.take()와 거의 유사한 인터페이스
draws = bag.take(sampler)
draws

array([-1, -1,  5,  6,  5,  4,  7,  7, -1,  6])

### 7.3.8 표시자/더미변수 (Computing indicator / dummy variables)
#### o 용도: 값의 종류에 따라 컬럼을 만들고, 그 값의 존재에 따라 1/0을 설정
#### o pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False)
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html
- Convert categorical variable into dummy/indicator variables
 - data : array-like, Series, or DataFrame
 - prefix : string, list of strings, or dict of strings, default None
   - String to append DataFrame column names

In [290]:
df = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                'data1': range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [293]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [294]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [300]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ch02/movielens/movies.dat', sep='::', header=None,names=mnames,engine='python')
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [ ]:
#### o str.split(str="", num=string.count(str))
- https://www.tutorialspoint.com/python/string_split.htm
- returns a list of all the words in the string, using parameter str as the separator

In [302]:
genre_iter = (set(x.split('|')) for x in movies.genres)
genres = sorted(set.union(*genre_iter))
genres

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [304]:
dummies = DataFrame(np.zeros((len(movies), len(genres))), columns=genres)
dummies

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [311]:
#확인: movies.genres
# movies.genres[1].split('|')
for i, gen in enumerate(movies.genres):
    dummies.ix[i, gen.split('|')] = 1
# 결과 확인
# dummies
# dummies.add_prefix('Genre_')

In [309]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.ix[0]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Action                                   0
Genre_Adventure                                0
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Crime                                    0
Genre_Documentary                              0
Genre_Drama                                    0
Genre_Fantasy                                  0
Genre_Film-Noir                                0
Genre_Horror                                   0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Romance                                  0
Genre_Sci-Fi                                   0
Genre_Thriller                                 0
Genre_War                                      0
Genre_Western       

#### o 통계에 활용: pd.cut()과 get_dummies() 이용한 통계 데이터 활용

In [315]:
np.random.seed(12345)

In [316]:
values = np.random.rand(10)
values

array([0.9296, 0.3164, 0.1839, 0.2046, 0.5677, 0.5955, 0.9645, 0.6532,
       0.7489, 0.6536])

In [317]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## 7.4 String manipulation
#### o 텍스트 연산의 대부분은 문자열 객체의 내장함수를 이용해 처리 가능
#### o 보다 복잡한 패턴매칭은 정규식(regular expression)활용 - pandas는 배열에 RE 적용 가능

### 7.4.1 String object methods
- https://docs.python.org/2.7/library/string.html#string-functions
- https://www.programiz.com/python-programming/methods/string

In [318]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

In [319]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [320]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [321]:
'::'.join(pieces)

'a::b::guido'

In [322]:
'guido' in val

True

In [323]:
val.index(',')

1

In [327]:
#string.find(): 없을 경우 -1 return
val.find(':')

-1

In [328]:
#string.index(): 없을 경우 에러 발생
val.index(':')

ValueError: substring not found

In [329]:
val.count(',')

2

In [330]:
val.replace(',', '::')

'a::b::  guido'

In [331]:
val.replace(',', '')

'ab  guido'

### 7.4.2 Regular expressions
#### o 파이썬에서는 re 모듈이 내장
- 학습: 
 - http://highcode.tistory.com/6
 - http://www.nextree.co.kr/p4327/
- 참고
 - "RegEx Learn the Hard way"는 유료화 
 - https://docs.python.org/3/library/re.html 

In [332]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

- re.compile() 결과를 활용하는 것이, computing power 소모 감소 

In [333]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

#### o Regular Expression Objects
- https://docs.python.org/3/library/re.html#regex-objects   - 6.2.3. Regular Expression Objects
- regex.findall(string[, pos[, endpos]])
  - 매치되는 모든 문자열을 찾아 줌.
  - Similar to the findall() function, using the compiled pattern, but also accepts optional pos and endpos parameters that limit the search region like for search()  
- regex.search(string[, pos[, endpos]])
 - 매치되는 첫번째 문자열을 찾아줌
 - pos: 매칭을 시작하는 문자열위치(0을 시작기준으로), endpoos: 매칭종료하는 문자열위치
- regex.match(string[, pos[, endpos]])
 - 시작되는 부분에서 매치되는 첫번째 문자열을 찾아줌

In [334]:
regex.findall(text)

['    ', '\t ', '  \t']

In [336]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [337]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [338]:
m = regex.search(text)
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [339]:
text[m.start():m.end()]

'dave@google.com'

In [340]:
print(regex.match(text))

None


In [341]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [342]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [344]:
# 위 URL (https://docs.python.org/3/library/re.html#regex-objects )의 
# 6.2.4. Match Objects 참고 
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [345]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [347]:
# \1 \2 \3 print문과 같이 text를 활용:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



In [348]:
regex = re.compile(r"""
    (?P<username>[A-Z0-9._%+-]+)
    @
    (?P<domain>[A-Z0-9.-]+)
    \.
    (?P<suffix>[A-Z]{2,4})""", flags=re.IGNORECASE|re.VERBOSE)

In [349]:
m = regex.match('wesm@bright.net')
m.groupdict()

{'domain': 'bright', 'suffix': 'net', 'username': 'wesm'}

### 7.4.3 Vectorized string functions in pandas

In [ ]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = Series(data)

In [ ]:
data

In [ ]:
data.isnull()

In [ ]:
data.str.contains('gmail')

In [ ]:
pattern

In [ ]:
data.str.findall(pattern, flags=re.IGNORECASE)

In [ ]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

In [ ]:
matches.str.get(1)

In [ ]:
matches.str[0]

In [ ]:
data.str[:5]

## 7.5 HomeWork: Example: USDA Food Database
### 

In [ ]:
import json
db = json.load(open('ch07/foods-2011-10-03.json'))
len(db)

In [ ]:
db[0].keys()

In [ ]:
db[0]['nutrients'][0]

In [ ]:
nutrients = DataFrame(db[0]['nutrients'])
nutrients[:7]

In [ ]:
info_keys = ['description', 'group', 'id', 'manufacturer']
info = DataFrame(db, columns=info_keys)

In [ ]:
info[:5]

In [ ]:
info

In [ ]:
pd.value_counts(info.group)[:10]

In [ ]:
nutrients = []

for rec in db:
    fnuts = DataFrame(rec['nutrients'])
    fnuts['id'] = rec['id']
    nutrients.append(fnuts)

nutrients = pd.concat(nutrients, ignore_index=True)

In [ ]:
nutrients

In [ ]:
nutrients.duplicated().sum()

In [ ]:
nutrients = nutrients.drop_duplicates()

In [ ]:
col_mapping = {'description' : 'food',
               'group'       : 'fgroup'}
info = info.rename(columns=col_mapping, copy=False)
info

In [ ]:
col_mapping = {'description' : 'nutrient',
               'group' : 'nutgroup'}
nutrients = nutrients.rename(columns=col_mapping, copy=False)
nutrients

In [ ]:
ndata = pd.merge(nutrients, info, on='id', how='outer')

In [ ]:
ndata

In [ ]:
ndata.ix[30000]

In [ ]:
result = ndata.groupby(['nutrient', 'fgroup'])['value'].quantile(0.5)
result['Zinc, Zn'].order().plot(kind='barh')

In [ ]:
by_nutrient = ndata.groupby(['nutgroup', 'nutrient'])

get_maximum = lambda x: x.xs(x.value.idxmax())
get_minimum = lambda x: x.xs(x.value.idxmin())

max_foods = by_nutrient.apply(get_maximum)[['value', 'food']]

# make the food a little smaller
max_foods.food = max_foods.food.str[:50]

In [ ]:
max_foods.ix['Amino Acids']['food']